<a href="https://colab.research.google.com/github/Mina-Gharenazi/Crossed-out-Puzzle-Solver/blob/main/Vulnerability_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# 1️⃣ Install nbformat
!pip install nbformat

# 2️⃣ Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 3️⃣ Load, clean, and overwrite your notebook file
import nbformat
import os

# Adjust this to the path in your Drive where the notebook lives
nb_path = '/content/drive/MyDrive/Colab Notebooks/Vulnerability_Detection.ipynb'

# Read as v4
nb = nbformat.read(nb_path, as_version=4)

# Remove any broken widget metadata
for cell in nb.cells:
    if 'widgets' in cell.metadata:
        del cell.metadata['widgets']

# Write it back
nbformat.write(nb, nb_path)
print("✅ Cleaned widget metadata in:", nb_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Cleaned widget metadata in: /content/drive/MyDrive/Colab Notebooks/Vulnerability_Detection.ipynb


# Tokens and Embediings

First Milestone covers:
   * Using the tokenizer "CodeBERT".
   * Getting the embeddings for functions and lines in each functions


In [ ]:
!pip install torch # for Model inference
!pip install transformers # for Tokenization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from transformers import RobertaTokenizer, RobertaModel
import torch
import json

In [ ]:
# Load the CodeBERT tokenizer and model from Hugging Face
# The tokenizer: breaks the code into tokens.
# The model: extracts vector embeddings from those tokens.
# CodeBERT is based on the RoBERTa architecture and trained on source code.
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Load JSON file (is named 'data.json' and located in google drive)

path_to_data= '/content/drive/MyDrive/Colab Notebooks/Big-Vul-dataset/data.json'
with open(path_to_data,'r') as file:
  data=json.load(file)
# printing the first two entries from your JSON dataset.
print(data[:2])

[{'code': 'static inline void clgi(void)\n{\n  asm volatile (__ex(SVM_CLGI));\n}\n', 'vul': 0, 'flaw_line_no': [], 'bigvul_id': 37737}, {'code': 'int nfs_open(struct inode *inode, struct file *filp)\n{\n  struct nfs_open_context *ctx;\n  struct rpc_cred *cred;\n  cred = rpc_lookup_cred();\n  if (IS_ERR(cred))\n    return PTR_ERR(cred);\n  ctx = alloc_nfs_open_context(filp->f_path.mnt, filp->f_path.dentry, cred);\n  put_rpccred(cred);\n  if (ctx == NULL)\n    return -ENOMEM;\n  ctx->mode = filp->f_mode;\n  nfs_file_set_open_context(filp, ctx);\n  put_nfs_open_context(ctx);\n  return 0;\n}\n', 'vul': 0, 'flaw_line_no': [], 'bigvul_id': 22804}]


In [ ]:
code_snippet=data[0]['code']
print(code_snippet)
input_ids = tokenizer.tokenize(code_snippet)
print(input_ids)

static inline void clgi(void)
{
  asm volatile (__ex(SVM_CLGI));
}

['static', 'Ġinline', 'Ġvoid', 'Ġcl', 'gi', '(', 'void', ')', 'Ċ', '{', 'Ċ', 'Ġ', 'Ġas', 'm', 'Ġvolatile', 'Ġ(', '__', 'ex', '(', 'S', 'VM', '_', 'CL', 'GI', '));', 'Ċ', '}', 'Ċ']


Ġ
This means there's a space before the token.
For example:
'Ġinline' = " inline"
'Ġvoid' = " void"
RoBERTa uses this instead of stripping spaces.
Ċ
This means newline (\n).
It helps the model understand code structure and indentation, which are important in programming.
This helps the model:
Understand where functions, blocks, and statements start/end.
Learn patterns in code like if (...) or { ... }.
Handle both keywords and identifiers properly, even if the identifier is new or rare.

In [ ]:
def tokenize_code(code_snippet, tockenizer,max_limit=512):
  '''A function that prepares each snippet for embedding generation'''
  input_ids=tockenizer.encode_plus( #function that tokenizes the input and adds details like padding and special tokens
      code_snippet, # input code
      max_length=max_limit, #max limit of 512 tokens
      add_special_tokens=True, # adds [CLS] at the start and [SEP] at the end
      padding='max_length', #makes sure all the inputs are the same length 512 tokens
      truncation=True, # cuts off token if they exceed 512
      return_tensors='pt') #converts to pytorch tensors for codeBERT
  return input_ids

print(tokenize_code(code_snippet,tokenizer)['input_ids'].shape)
print(tokenize_code(code_snippet,tokenizer))

torch.Size([1, 512])
{'input_ids': tensor([[    0, 42653, 39491, 13842,  3741, 15696,  1640, 47908,    43, 50118,
         45152, 50118,  1437,    25,   119,  9234,    36, 30529,  3463,  1640,
           104, 20954,  1215,  7454, 21525, 48749, 50118, 24303, 50118,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,  

It sends the tokenized code into the CodeBERT model and gets its embeddings (hidden states).

In [ ]:
def gen_embedings(token,model): #get embeddings from CodeBERT
  """ Generate embeddings for each token"""
  with torch.no_grad():         # we don't want to train the model now and we need just embeddings. so in training we should remove it
    outputs=model(**token)
  return outputs

In [ ]:
# Tokenize json with tockenize_code in a loop
data_tokenized=[]
function_embeddings=[]
embeddings_data=[]
for item in data[:20]:
  code_snippet=item['code']
  tokens=tokenize_code(code_snippet,tokenizer)
  embeddings=gen_embedings(tokens,model)
  data_tokenized.append(
      {
  "bigvul_id": item["bigvul_id"],
  'Tokens': tokens,
  "vul":item["vul"]
      }
  )
  function_embeddings.append(
      {
          # "bigvul_id": item["bigvul_id"],
          "vul":item["vul"],
          "embeddings": embeddings
      }
  )
  # Line-level embeddings (128 tokens)
  lines_data = []
  lines = code_snippet.strip().split('\n')
  flaw_line_no = set(item["flaw_line_no"])
  for idx, line in enumerate(lines, 1):
      tokens=tokenize_code(line,tokenizer, 128)
      line_embedding = gen_embedings(tokens,model)
      lines_data.append({
          "line_no": idx,
          "code": line,
          "embeddings": line_embedding,
          "is_flawed": idx in flaw_line_no
      })

  #The following line was not indented correctly, causing the error.
  embeddings_data.append({ #Fixed indentation
        "bigvul_id": item["bigvul_id"],
        "function_embedding": function_embeddings[-1], #Use the latest function embedding
        "line_embedding": lines_data
    })


# print(function_embeddings(data)[0]['embeddings'].pooler_output.shape)
# print(function_embeddings(data)[0]['embeddings'].last_hidden_state.shape)
# print(function_level_embeddings(data)[0]['embeddings'].last_hidden_state)
# print(function_level_embeddings(data)[0]['embeddings'].pooler_output.shape)
# print(dir(function_level_embeddings(data)[0]['embeddings']))
# print(function_level_embeddings(data))
# print(data_tokenized[:1])

In [ ]:
# print(embeddings_data)
# print(embeddings_data[0]['line_embedding'][0]['embeddings'])
print(embeddings_data[0]['line_embedding'][0])

{'line_no': 1, 'code': 'static inline void clgi(void)', 'embeddings': BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.0656e-01,  2.1345e-01,  4.2046e-02,  ..., -1.5824e-01,
          -3.1536e-01,  2.8361e-01],
         [-3.5546e-01,  7.9650e-02,  4.1705e-01,  ..., -1.3414e+00,
          -2.0669e-01,  8.6232e-01],
         [-1.4033e-01, -1.2040e-01,  3.5286e-01,  ...,  8.7561e-02,
          -1.6834e-01,  3.4170e-01],
         ...,
         [ 2.4197e-01,  1.9250e-01,  2.1090e-01,  ..., -8.9889e-04,
          -3.4281e-01,  5.7012e-02],
         [ 2.4197e-01,  1.9250e-01,  2.1090e-01,  ..., -8.9889e-04,
          -3.4281e-01,  5.7012e-02],
         [ 2.4197e-01,  1.9250e-01,  2.1090e-01,  ..., -8.9889e-04,
          -3.4281e-01,  5.7012e-02]]]), pooler_output=tensor([[ 3.9794e-01, -4.1409e-01, -5.5573e-01,  1.8097e-01,  3.7038e-01,
          6.9104e-03,  4.8677e-01, -3.2477e-01,  4.7598e-02, -1.9455e-01,
          3.6667e-01,  8.6304e-03, -3.0196e-01,  1.4109e-0

In [ ]:
print(torch.cuda.is_available())

NameError: name 'torch' is not defined